In [ ]:
# import pdb; pdb.set_trace()
# import pudb; pudb.set_trace()

# df.style.background_gradient(
#     cmap='coolwarm', 
#     axis=None, 
#     vmin=-1, 
#     vmax=1
# )  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG'

# pd.set_option('precision', 2)

# .plot_bokeh(sizing_mode="scale_height")
# .plot_bokeh(kind='hist', bins=1000, sizing_mode="scale_height", xlim=[0, .1])

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 99)

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

import datetime as dt
import pickle
import numpy as np
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_file("tmp/bokeh_output.html")
import ray

from data_layer import arrow_dataset, storage_adaptor
from tick_filter import streaming_tick_filter
from tick_sampler import streaming_tick_sampler, daily_stats
from workflows import sampler_task, sampler_flow, syncbar_flow, syncbar_task
from workflows.configs import renko_v1, renko_v2
from utilities import date_fu, project_globals as g
from data_layer import storage_adaptor, fsspec_factory, data_access, arrow_dataset

In [1]:
!pip install tenacity

Defaulting to user installation because normal site-packages is not writeable


In [2]:
config = renko_v2.config

prefix_data = '/data/trades'

prefix_meta = f"/bars/{config['meta']['config_id']}/meta"

prefix_df = f"/bars/{config['meta']['config_id']}/df"

# Ray setup

In [3]:
ray.init(dashboard_host='0.0.0.0', dashboard_port=1111, ignore_reinit_error=True)

2021-06-10 20:46:42,458	INFO services.py:1272 -- View the Ray dashboard at http://10.0.0.4:1111


{'node_ip_address': '10.0.0.4',
 'raylet_ip_address': '10.0.0.4',
 'redis_address': '10.0.0.4:6379',
 'object_store_address': '/tmp/ray/session_2021-06-10_20-46-41_476782_13808/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-06-10_20-46-41_476782_13808/sockets/raylet',
 'webui_url': '10.0.0.4:1111',
 'session_dir': '/tmp/ray/session_2021-06-10_20-46-41_476782_13808',
 'metrics_export_port': 56375,
 'node_id': '08e2de54d5b5e0e74ccde5f6013211952f45c4b6365cd9ca341a0736'}

In [ ]:
ray.shutdown()

# sync bars

In [4]:
config = {
    'config_id': 'renko_v2',
    'start_date': '2019-01-01',
    'end_date': '2021-01-01',
    'source': 'remote',
    'destination': 'both',
    'on_ray': True,
}

In [ ]:
syncbar_flow.run(config)

syncing symbols: AGI AEM
scheduled 0 dates
syncing symbols: AGI FNV
scheduled 0 dates
syncing symbols: AGI GLD
scheduled 0 dates
syncing symbols: AGI KGC
scheduled 0 dates
syncing symbols: AGI KL
scheduled 0 dates
syncing symbols: AGI NEM
scheduled 0 dates
syncing symbols: AGI VTI
scheduled 0 dates
syncing symbols: AGI WPM
scheduled 0 dates
syncing symbols: BVN AEM
scheduled 0 dates
syncing symbols: BVN FNV
scheduled 0 dates
syncing symbols: BVN GLD
scheduled 0 dates
syncing symbols: BVN KGC
scheduled 0 dates
syncing symbols: BVN KL
scheduled 0 dates
syncing symbols: BVN NEM
scheduled 0 dates
syncing symbols: BVN VTI
scheduled 0 dates
syncing symbols: BVN WPM
scheduled 0 dates
syncing symbols: EXK AEM
scheduled 0 dates
syncing symbols: EXK FNV
scheduled 0 dates
syncing symbols: EXK GLD
scheduled 0 dates
syncing symbols: EXK KGC
scheduled 0 dates
syncing symbols: EXK KL
scheduled 0 dates
syncing symbols: EXK NEM
scheduled 0 dates
syncing symbols: EXK VTI
scheduled 243 dates
syncing symb

# Tick Sampler 

In [ ]:
bds = sampler_flow.run(config)

In [ ]:
n = 1
print(bds[n]['date'])
# bds[n]['ticks_df'][['price', 'price_jma']].plot_bokeh(sizing_mode="scale_height")
bds[n]['ticks_df'][['price', 'price_jma']].plot()

In [ ]:
bds[n]['ticks_df'][['price_jma']].plot()

In [ ]:
bds[n]['ticks_df'].status.value_counts() / len(bds[n]['ticks_df'])

In [ ]:
bds[n]['bars_df'][['price_low', 'price_high', 'price_close']].plot()

# Price gap fill

In [ ]:
from sample_features import stacked
# fill daily gaps
stacked_df = stacked.fill_gaps_dates(bar_dates, fill_col='price_vwap')

# stacked_df[['price_high','price_low','price_close', 'price_vwap']].plot_bokeh(sizing_mode="scale_height")

# stacked stats
stats_df = stacked.stacked_df_stats(stacked_df)


In [ ]:
stats_df.describe()